# Sample code how to use `logllm` package

## Your (messy) machine learning code is here

In [1]:
from sklearn import datasets as ds
from sklearn.model_selection import train_test_split as tts
from sklearn.svm import SVC as SupportVectorClassifier
from sklearn.metrics import confusion_matrix as cm, classification_report as cr
import numpy as np

# データセットのロード
iris_dataset = ds.load_iris()

# クラスラベルを二値化（クラス2を除外）
features = iris_dataset.data[iris_dataset.target != 2]  
labels = iris_dataset.target[iris_dataset.target != 2]  

# トレーニングデータとテストデータに分割
train_data, test_data, train_labels, test_labels = tts(
    features, 
    labels, 
    test_size=0.2, 
    random_state=42, 
    shuffle=True, 
    stratify=labels  
)

# サポートベクターマシンモデルの初期化と設定
svc_model = SupportVectorClassifier(
    C=1.0,        
    kernel='linear', 
    degree=3,     
    gamma='auto', 
    coef0=0.0,    
    shrinking=True,
    probability=False, 
    tol=1e-3,     
    cache_size=200, 
    class_weight=None,
    verbose=False, 
    max_iter=-1,  
    decision_function_shape='ovr',
    break_ties=False, 
    random_state=None
)

# モデルのトレーニング
svc_model.fit(train_data, train_labels)

# テストデータに対する予測結果の取得
predicted_labels = svc_model.predict(test_data)

# 混同行列の作成
conf_matrix = cm(test_labels, predicted_labels)

# 評価用ラベルの定義（クラス1をわざと除外）
excluded_class = 1
classes_to_evaluate = [cls for cls in np.unique(test_labels) if cls != excluded_class]

# 特定クラスを除外した混同行列の計算
conf_matrix_filtered = np.array([
    [conf_matrix[i, j] for j in range(len(conf_matrix)) if j != excluded_class]
    for i in range(len(conf_matrix)) if i != excluded_class
])

# 正解ラベルと予測ラベルをフィルタリング（評価対象クラスのみ残す）
filtered_test_labels = [label for label in test_labels if label in classes_to_evaluate]
filtered_predicted_labels = [pred for pred in predicted_labels if pred in classes_to_evaluate]

# 詳細な分類レポートの出力（特定クラスを除外）
classification_report_filtered = cr(
    filtered_test_labels, 
    filtered_predicted_labels, 
    labels=classes_to_evaluate,
    target_names=[f"Class {cls}" for cls in classes_to_evaluate],
    digits=4,
    output_dict=False,
    zero_division=0
)

# 複雑な評価結果の表示
print(f"Filtered Confusion Matrix (excluding Class {excluded_class}):\n{conf_matrix_filtered}")
print("\nDetailed Classification Report (excluding certain classes):")
print(classification_report_filtered)

# 意図的に除外したクラスに関する補足情報
if excluded_class in np.unique(test_labels):
    print(f"\nNote: Class {excluded_class} was intentionally excluded from the evaluation.")

Filtered Confusion Matrix (excluding Class 1):
[[10]]

Detailed Classification Report (excluding certain classes):
              precision    recall  f1-score   support

     Class 0     1.0000    1.0000    1.0000        10

    accuracy                         1.0000        10
   macro avg     1.0000    1.0000    1.0000        10
weighted avg     1.0000    1.0000    1.0000        10


Note: Class 1 was intentionally excluded from the evaluation.


## Start logging with `logllm`

In [2]:
from logllm import log_llm

notebook_path = "svc-sample.ipynb" # Here is target file to log

log_llm(notebook_path,is_logging=False)

{
    "method": "SVC",
    "dataset": "Iris",
    "task": "classification",
    "accuracy": 1.0,
    "C": 1.0,
    "degree": 3,
    "tol": 0.001,
    "cache_size": 200,
    "max_iter": -1,
    "test_size": 0.2,
    "random_state": 42,
    "kernel": "linear",
    "condition_as_natural_langauge": [
        "Using linear kernel on SVC model.",
        "Excluding class 2 from Iris dataset.",
        "Splitting data into 80% training and 20% testing."
    ],
    "advice_to_improve_acc": [
        "Consider using cross-validation for better performance evaluation.",
        "Experiment with different kernels to optimize results.",
        "Increase the dataset size to improve generalization."
    ]
}
Response from OpenAI logged to W&B.


## Expected condition with GPT4 from the script

```Python
{
    "method": "SVC",
    "dataset": "Iris",
    "task": "classification",
    "accuracy": 1.0,
    "C": 1.0,
    "degree": 3,
    "tol": 0.001,
    "cache_size": 200,
    "max_iter": -1,
    "test_size": 0.2,
    "random_state": 42,
    "kernel": "linear",
    "condition_as_natural_langauge": [
        "Using linear kernel on SVC model.",
        "Excluding class 2 from Iris dataset.",
        "Splitting data into 80% training and 20% testing."
    ],
    "advice_to_improve_acc": [
        "Consider using cross-validation for better performance evaluation.",
        "Experiment with different kernels to optimize results.",
        "Increase the dataset size to improve generalization."
    ]
}

```